In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import numpy as np
import pandas as pd

In [7]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [11]:
df = pd.read_csv('/content/drive/MyDrive/day28-Column-transformer/covid_toy.csv')
df.shape

(100, 6)

In [12]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [13]:
X = df.drop(columns=['has_covid'])
y = df['has_covid']

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [17]:
X_train

,age,gender,fever,cough,city
3,31,Female,98.0,Mild,Kolkata
85,16,Female,103.0,Mild,Bangalore
56,71,Male,NaN,Strong,Kolkata
23,80,Female,98.0,Mild,Delhi
51,11,Female,100.0,Strong,Kolkata
...,...,...,...,...,...
43,22,Female,99.0,Mild,Bangalore
11,65,Female,98.0,Mild,Mumbai
33,26,Female,98.0,Mild,Kolkata
36,38,Female,101.0,Mild,Bangalore


## 1. Aam Zindagi (Without Column transformer)

In [19]:
#adding simple imputer to fever column
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [21]:
df.columns

Index(['age', 'gender', 'fever', 'cough', 'city', 'has_covid'], dtype='object')

In [23]:
X_train['cough'].value_counts()

,count
cough,
Mild,51
Strong,29


In [24]:
# Ordinal encoding -> cough
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [26]:
# One-Hot Encoding -> gender, city columns
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [27]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [32]:
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough), axis=1)

# also the test data
X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough), axis=1)

X_train_transformed.shape

(80, 7)

In [36]:
pd.DataFrame(X_train_transformed, columns=['age', 'fever', 'gc01', 'gc02', 'gc03', 'gc04', 'cough'])

,age,fever,gc01,gc02,gc03,gc04,cough
0,31.0,98.000000,0.0,0.0,1.0,0.0,0.0
1,16.0,103.000000,0.0,0.0,0.0,0.0,0.0
2,71.0,100.887324,1.0,0.0,1.0,0.0,1.0
3,80.0,98.000000,0.0,1.0,0.0,0.0,0.0
4,11.0,100.000000,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...
75,22.0,99.000000,0.0,0.0,0.0,0.0,0.0
76,65.0,98.000000,0.0,0.0,0.0,1.0,0.0
77,26.0,98.000000,0.0,0.0,1.0,0.0,0.0
78,38.0,101.000000,0.0,0.0,0.0,0.0,0.0
